In [93]:
# --- Matrix Class --- 

import math

from Array import Array2D

class Matrix(Array2D):

    def __init__(self,numRows,numCols):
        super().__init__(numRows,numCols)
        self._shape = numRows , numCols

    # Multiple Scalar 
    def scaleBy(self,scalar):

        newMatrix = Matrix(self.numRows() , self.numCols())
        for i in range( self.numRows()):
            for j in range( self.numCols()):
                newMatrix[i,j] = self[i,j] * scalar
        return newMatrix

    # Square all data
    def square(self):
        newMatrix = Matrix(self.numRows(), self.numCols())

        for i in range(self.numRows()):
            for j in range(self.numCols()):
                newMatrix[i,j] = self[i,j] * self[i,j]
        return newMatrix

    def log(self):
        newMatrix = Matrix(self.numRows(), self.numCols())
        for i in range(self.numRows()):
            for j in range(self.numCols()):
                newMatrix[i,j] = math.log(self[i,j])
        return newMatrix
    
    def sqrt(self):
        newMatrix = Matrix(self.numRows(), self.numCols())
        for i in range(self.numRows()):
            for j in range(self.numCols()):
                newMatrix[i,j] = math.sqrt(self[i,j])
        return newMatrix
    


    def sum(self):
        total = 0
        for i in range(self.numRows()):
            for j in range(self.numCols()):
                total += self[i,j]

        return total

    # Transpose สลับแถวกับหลัก
    def transpose(self):
        newMatrix = Matrix(self.numCols() , self.numRows())
        for i in range( self.numRows()):
            for j in range( self.numCols()):
                newMatrix[j,i] = self[i,j]
        return newMatrix

    # Addition
    def __add__ ( self , rhsMatrix ) :
        assert rhsMatrix.numRows() == self.numRows() and rhsMatrix.numCols() == self.numCols() ,\
            "Matrix size not compatible for the add operation."
        newMatrix = Matrix( self.numRows() , self.numCols() )
        for i in range( self.numRows() ):
            for j in range( self.numCols() ) :
                newMatrix[i,j] = self[i,j] + rhsMatrix[i,j]
        return newMatrix
        
    # Subtraction
    def __sub__ ( self , rhsMatrix ) :
        assert rhsMatrix.numRows() == self.numRows() and rhsMatrix.numCols() == self.numCols() ,\
            "Matrix size not compatible for the add operation."
        newMatrix = Matrix( self.numRows() , self.numCols() )
        for i in range( self.numRows() ):
            for j in range( self.numCols() ) :
                newMatrix[i,j] = self[i,j] - rhsMatrix[i,j]
        return newMatrix
    
    # Element-wise Multiplication
    def __mul__ ( self , rhsMatrix):
        assert rhsMatrix.numRows() == self.numRows() and rhsMatrix.numCols() == self.numCols() ,\
            "Matrix size not compatible for the add operation."
        newMatrix = Matrix( self.numRows(), self.numCols())
        for i in range( self.numRows() ):
            for j in range( self.numCols() ) :
                newMatrix[i,j] = self[i,j] * rhsMatrix[i,j]
        return newMatrix    

    # Dot Product (Matrix Multiplication)
    def dotproduct(self,rhsMatrix):
        assert self.numCols() == rhsMatrix.numRows() , "Cannot multiply these matrices."
        newMatrix = Matrix( self.numRows() , rhsMatrix.numCols())
        for i in range(self.numRows()):
            for j in range(rhsMatrix.numCols()):
                total = 0
                for k in range(rhsMatrix.numRows()):
                     total += self[i,k] * rhsMatrix[k,j]
                newMatrix[i,j] = total
        return newMatrix

    #Load list
    def loadList(self,data):
        # Check that they has 1 row or more
        try: 
            data_column = len(data[0])
            data_row = len(data)
            assert (self.numCols() == data_column and self.numRows() == data_row) , "incorrect size to load data to matrix."
            for i in range(data_row):
                for j in range(data_column):
                    self[i,j] = data[i][j]

        except: # For the 1 row
            for i in range(len(data)):
                self[0,i] = data[i]
                


    def reshape(self,row,col):
        
        # เช็คว่าขนาดในการ Reshape เท่ากันไหม
        assert (row * col == self.numRows() * self.numCols()) , "Size couldn't fit in the Array2D"
        
        # 1 Solution - เข้าถึง matrix ก่อน แล้วดึงเข้า list เปล่า
        number_list = []
        for i in range(self.numRows()):
            for j in range(self.numCols()):
                number_list.append(self[i,j])

        # 2 Solution - เข้าถึง Matrix รูปร่างใหม่ แล้ว Assign ค่าลงไปผ่าน list เดี่ยว (ใช้ index บวกไปเรื่อยๆ)
        
        newMatrix = Array2D(row,col)
        ndx = 0

        for i in range(row):
            for j in range(col):
                newMatrix[i,j] = number_list[ndx]
                ndx += 1

        return newMatrix
    
    # Load Function - Let every data in hidden goes through the activate function.
    def loadFunction(self,activate_function):        
        for i in range(self.numRows()):
            for j in range(self.numCols()):
                self[i,j] = activate_function(self[i,j])

    # Average - compute the weighted average of all the data
    def average(self):
        total = 0
        for i in range(self.numRows()):
            for j in range(self.numCols()):
                total += self[i,j]

        average = total / (self.numRows() * self.numCols())
        return average

In [94]:
# --- ANN Model Class ---
import random
import math
import statistics

class ANN_Model():
    def __init__(self,file): 
        with open(file,'r') as f:
            self._RawData = f.readlines()

    def cleanData(self):

        newData = []
        resultData = []
        for row in self._RawData:
            check_row = row.split(',')
            last = check_row.pop()
            resultData.append(last[0])


            check_row = [float(num) for num in check_row]

            # Column
            current_row = []
            for column in range(len(check_row)):
                if (column != 1):
                    current_row.append(check_row[column])
  
            newData.append(current_row)

        # self.Normalization(newData,resultData)
        self._RawInputData = newData

        # result Data is ['g','b','g',...]
        rawtargetdata = [list(result) for result in resultData]
        ## We can detect it [1,0] ,[0,1]

        self._RawTargetData = rawtargetdata

        rawCleanInputData = []
        rawCleanTargetData = []

        # Remove the row that is the suspicious data
        for i in range(len(self._RawInputData)):

            rawCleanInputData.append(self._RawInputData[i][0:])
            rawCleanTargetData.append(self._RawTargetData[i])
                
        self._RawCleanInputData = rawCleanInputData
        self._RawCleanTargetData = rawCleanTargetData
        
        # Create instant variables for hyperparameter
        X = self._RawCleanInputData
        y = self._RawCleanTargetData

        self._RawCleanData = []

        # Merge and write it in the csv file to check

        for i in range(len(self._RawCleanInputData)):
            self._RawCleanData.append(self._RawCleanInputData[i] + self._RawCleanTargetData[i])


        # Sorting by the output
        self._RawCleanData = sorted(self._RawCleanData, key=lambda x: x[-1])

        import csv

        with open('rawcleandata.csv' , 'w' , encoding="UTF-8" , newline="") as f:
            writer = csv.writer(f)

            # write multiple rows
            writer.writerows(self._RawCleanData)

        self._NumData = len(X)
        self._NumInput = len(X[0])
        self._NumOutput = len(y[0])

        # after sorting the data spreading the raw data again

        self._RawCleanTargetData = []
        self._RawCleanInputData = []

        for column in self._RawCleanData:
            self._RawCleanInputData.append(column[:-1])

            if column[-1] == "g":
                self._RawCleanTargetData.append(1)
            elif column[-1] == "b":
                self._RawCleanTargetData.append(0)

        # If you are not wish to use StandardScaler or MinMaxScaler

        self._ScaledTrainData = self._RawCleanInputData
        self._ScaledTargetData = self._RawCleanTargetData

  # Data Spliting
    def splitData(self,test_size = 0.8):

        self._Train_X , self._Train_y , self._Test_X , self._Test_y = [] , [] , [] , []

        b_data = []
        g_data = []

        # Train and Test Data should be distribute at same
        for i in range(351):
            if i < 126:
                b_data.append(self._ScaledTrainData[i] + [ self._ScaledTargetData[i] ])
            else:
                g_data.append(self._ScaledTrainData[i] + [ self._ScaledTargetData[i] ])
                
        random.shuffle(b_data)
        random.shuffle(g_data)

        X_b , y_b , X_g , y_g = [] , [] , [] , []

        # Since the data is the imbalanced classification so we will make sure that
        # the minor class will have a same distribution in train and test data

        # Bad Section
        
        for i in range(len(b_data)): 
            X_b.append(b_data[i][:-1])
            y_b.append(b_data[i][-1])

        # split to train and test equally

        for i in range(len(b_data)):
            if i % 3 == 0:
                self._Test_X.append(X_b[i])
                self._Test_y.append(y_b[i])
            else:
                self._Train_X.append(X_b[i])
                self._Train_y.append(y_b[i])

        # Good Section
        for i in range(len(g_data)):
            X_g.append(g_data[i][:-1])
            y_g.append(g_data[i][-1])

        # Split to train and test equally
        
        for i in range(len(g_data)):
            if i % 3 == 0:
                self._Test_X.append(X_g[i])
                self._Test_y.append(y_g[i])
            else:
                self._Train_X.append(X_g[i])
                self._Train_y.append(y_g[i])


        # Shuffle the Train and Test Data
        
        Train_dataset , Test_dataset = [] , []

        for i in range(len(self._Train_X)):
            Train_dataset.append(self._Train_X[i] + [self._Train_y[i]])

        for i in range(len(self._Test_X)):
            Test_dataset.append(self._Test_X[i] + [ self._Test_y[i] ])

        random.shuffle(Train_dataset)
        random.shuffle(Test_dataset)

        self._Train_X , self._Train_y , self._Test_X , self._Test_y = [] , [] , [] , []

        for i in range(len(Train_dataset)):
            self._Train_X.append(Train_dataset[i][:-1])
            self._Train_y.append(Train_dataset[i][-1])

        for i in range(len(Test_dataset)):
            self._Test_X.append(Test_dataset[i][:-1])
            self._Test_y.append(Test_dataset[i][-1])

        self._NumTrain = len(self._Train_X)
        self._NumTest = len(self._Test_X)

        Input_Train = Matrix(len(self._Train_X),len(self._Train_X[0]))
        Input_Train.loadList(self._Train_X)
        # Input in matrix form
        self._Input_Train = Input_Train

        Output_Train = Matrix(1,len(self._Train_y))
        Output_Train.loadList(self._Train_y)
        # Output in matrix form
        self._Output_Train = Output_Train

        Input_Test = Matrix(len(self._Test_X), len(self._Test_X[0]))
        Input_Test.loadList(self._Test_X)
        
        self._Input_Test = Input_Test

        Output_Test = Matrix(1, len(self._Test_y))
        Output_Test.loadList(self._Test_y)

        self._Output_Test = Output_Test

    def hyperparameter(self):
        # Rules of Thumbs
        self._NumHidden = int((2/3) * (self._NumInput + self._NumOutput))
        # self._NumHidden = numhidden
        print("No.Input Unit: ",self._NumInput)
        print("No.Hidden Layer: ", 1)
        print("No.Hidden Unit: ",self._NumHidden)
        print("No.Output Unit: ",self._NumOutput)
    
    def trainModel(self,random_state=None,epochs=50,alpha=0.01,batch_size=32):

        import math

        # กำหนด Random Seed
        if random_state is not None:
            random.seed(random_state)
            print("Random Seed :" , random_state)
        else: 
            random.seed(42)
            print("Random Seed :" , 42)

        # Random weights between input and hidden

        # He Normal Initialization

        self._weights_input_hidden = Matrix(self._NumInput, self._NumHidden)
        for i in range(self._NumInput):
            for j in range(self._NumHidden):
                sd = math.sqrt(2 / self._NumInput)
                self._weights_input_hidden[i, j] = random.normalvariate(0.0, sd)

        # # Random weights between hidden and output
        self._weights_hidden_output = Matrix(self._NumHidden, self._NumOutput)
        for i in range(self._NumHidden):
            for j in range(self._NumOutput):
                sd = math.sqrt(2 / self._NumHidden )
                self._weights_hidden_output[i, j] = random.normalvariate(0.0, sd)

        Input = self._Input_Train.transpose()
        Target = self._Output_Train

        losses = []

        # Sigmoid - Activate Function 
        def sigmoid(x): # หรือ Logistic Function
            return 1 / (1 + math.exp(-x))
        
        # ReLu - Activate Function
        def relu(x):
            return max(0, x)

        # Training Loop
        for epoch in range(epochs):

            random_list = [int(i) for i in range(self._NumTrain)]
            Train_seq = random.sample(random_list,self._NumTrain)

            # Batch size

            for start in range(0, self._NumTrain, batch_size):
                end = start + batch_size
                batch_indices = Train_seq[start:end]

                # Shuffle batch indices

                random.shuffle(batch_indices)

                for seq_num in batch_indices:

                    # Select Column ([:,Train_seq[i]])
                    input_selected_column = []
                    for i in range(Input.numRows()):
                        j = Train_seq[seq_num]
                        input_selected_column.append(Input[i,j])
                    
                    input_selected_column_Matrix = Matrix(self._NumInput,1)
                    for i in range(len(input_selected_column)):
                        input_selected_column_Matrix[i,0] = input_selected_column[i]
                    
                    # Feed Forward

                    Hidden = self._weights_input_hidden.transpose().dotproduct(input_selected_column_Matrix)
                    
                    # Relu Function
                    Hidden.loadFunction(relu)
                    

                    Output = self._weights_hidden_output.transpose().dotproduct(Hidden)
                    Output.loadFunction(sigmoid)

                    # --- Backpropagation Algorithm
                    
                    # Target Select Column - Target[:,Train_seq[seq_num]]
                    target_selected_column = []
                    for i in range(Target.numRows()):
                        j = Train_seq[seq_num]
                        target_selected_column.append(Target[i,j])

                    
                    target_selected_column_Matrix = Matrix(1,len(target_selected_column))
                    for i in range(len(target_selected_column)):
                        target_selected_column_Matrix[0,i] = target_selected_column[i]

                    target_selected_column_Matrix = target_selected_column_Matrix.square()

                    # Normal Error
                    # Error = target_selected_column_Matrix - Output
                    # Error.scaleBy(1/2)

                    oneminus_Output = Matrix(Output.numRows(),Output.numCols())
                    oneminus_Output.clear(1) 

                    # Binary Cross Entropy Loss
                    
                    loss = (
                    (target_selected_column_Matrix * Output.log()) + (
                    (oneminus_Output - target_selected_column_Matrix) * (oneminus_Output - Output).log()))
                    loss = loss.scaleBy(-1)

                    Error_wrt_output = target_selected_column_Matrix - Output
                    Error_wrt_output = Error_wrt_output.scaleBy(-1)

                    # oneminus_Output = 1 - Output

                    Output_wrt_outputLayer_LinearTransform = Output * (oneminus_Output - Output)
                    OutputLayer_LinearTransform_wrt_weights_hidden_output = Hidden
                    a = Error_wrt_output * Output_wrt_outputLayer_LinearTransform
                    a = a.transpose()
                    Error_wrt_weights_hidden_output = OutputLayer_LinearTransform_wrt_weights_hidden_output.dotproduct(a)
                    
                    # oneminus_Hidden = 1 - Hidden
                    oneminus_Hidden = Matrix(Hidden.numRows(),Hidden.numCols())
                    oneminus_Hidden.clear(1)

                    HiddenLayer_activations_wrt_hiddenLayer_linearTransform = Hidden * (oneminus_Hidden - Hidden)
                    OutputLayer_LinearTransform_wrt_hiddenLayer_activations = self._weights_hidden_output
                    HiddenLayer_linearTransform_wrt_weights_input_hidden = input_selected_column_Matrix
                    b = Output_wrt_outputLayer_LinearTransform * Error_wrt_output
                    b = OutputLayer_LinearTransform_wrt_hiddenLayer_activations.dotproduct(b)
                    b = HiddenLayer_activations_wrt_hiddenLayer_linearTransform * b
                    b = b.transpose()
                    Error_wrt_weights_input_hidden = HiddenLayer_linearTransform_wrt_weights_input_hidden.dotproduct(b)
                
                    # #Normal Updating weights term
                    self._weights_hidden_output = self._weights_hidden_output - Error_wrt_weights_hidden_output.scaleBy(alpha)
                    self._weights_input_hidden = self._weights_input_hidden - Error_wrt_weights_input_hidden.scaleBy(alpha)

            # epoch_loss = Error.average()
            epoch_loss = loss.sum()

            if epoch % 10 == 0:
                print(f"Error at epoch {epoch} is {epoch_loss:.5f}")

            losses.append(epoch_loss)

        # Visualizing the error
        import matplotlib.pyplot as plt
        arange = [int(i) for i in range(1,epochs+1)]
        plt.plot(arange, losses)

    def testModel(self):
        Input = self._Input_Test.transpose()
        Target = self._Output_Test

        def sigmoid(x): # หรือ Logistic Function
            return 1 / (1 + math.exp(-x))
        
        def relu(x):
            return max(0, x)

        # Forward Propogation

        Hidden = self._weights_input_hidden.transpose().dotproduct(Input)
        Hidden.loadFunction(relu)

        # Calculating the output
        Output = self._weights_hidden_output.transpose().dotproduct(Hidden)
        Output.loadFunction(sigmoid)

        Output_list = [Output[0, i] for i in range(Output.numCols())]
        Target_list = [Target[0,i] for i in range(Output.numCols())]
            
        import csv

        with open('output.csv' , 'w' , encoding="UTF-8" , newline="") as f:
            writer = csv.writer(f)
            writer.writerow(Output_list)
            writer.writerow(Target_list)

        min_threshold = min(Output_list)
        max_threshold = max(Output_list)

        accuracy_list = {}
        precision_list = {}
        f1_list = {}

        while min_threshold <= max_threshold:

            Class_Output = []
            Class_Target = []

            # Iterating through the output matrix (1 x numTest)

            Class_Output = [1 if value >= min_threshold else 0 for value in Output_list]
            Class_Target = [Target[0, j] for j in range(Target.numCols())]

            self._class_output = Class_Output
            self._class_target = Class_Target

            correct = 0
            miss = 0

            conf_matrix = [ [0,0] , [0,0]]
            for i in range(self._NumTest):
                if Class_Output[i] == Class_Target[i]:
                    correct += 1
                else:
                    miss += 1
                conf_matrix[Class_Target[i]][Class_Output[i]] += 1

            accuracy = correct / self._NumTest

            # Precision, Recall, and F1 Score
            tp = conf_matrix[1][1]
            fp = conf_matrix[0][1]
            fn = conf_matrix[1][0]

            precision = tp / (tp + fp) if tp + fp != 0 else 0
            recall = tp / (tp + fn) if tp + fn != 0 else 0
            f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall != 0 else 0

            accuracy_list[min_threshold] = accuracy
            precision_list[min_threshold] = precision
            f1_list[min_threshold] = f1_score

            min_threshold += 0.000001

        best_threshold = max(zip(accuracy_list.values() , accuracy_list.keys() ))[1]
        # best_threshold = max(zip(precision_list.values() , precision_list.keys()))[1]
        
        Class_Output = []
        Class_Target = []

        # Iterating through the output matrix (1 x numTest)

        Class_Output = [1 if value >= best_threshold else 0 for value in Output_list]
        Class_Target = [Target[0, j] for j in range(Target.numCols())]

        self._class_output = Class_Output
        self._class_target = Class_Target

        correct = 0
        miss = 0

        conf_matrix = [ [0,0] , [0,0]]
        for i in range(self._NumTest):
            if Class_Output[i] == Class_Target[i]:
                correct += 1
            else:
                miss += 1
            conf_matrix[Class_Target[i]][Class_Output[i]] += 1

        accuracy = (correct / self._NumTest)

        # Precision, Recall, and F1 Score
        tp = conf_matrix[1][1]
        fp = conf_matrix[0][1]
        fn = conf_matrix[1][0]

        precision = tp / (tp + fp) if tp + fp != 0 else 0
        recall = tp / (tp + fn) if tp + fn != 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall != 0 else 0

        print('Best Threshold is :' , best_threshold)
        print('Confusion Matrix (row=Actual, col=Predicted)')
        print(conf_matrix)
        print('Accuracy: {:.2f}%'.format(accuracy * 100))
        print('Precision: {:.2f}'.format(precision))
        print('Recall: {:.2f}'.format(recall))
        print('F1 Score: {:.2f}'.format(f1_score))



### 1. Python

In [95]:
import os

os.getcwd()

'c:\\Users\\folks\\Desktop\\cpe112-data-structure\\Final_Project'

In [96]:
# --- 2. Data preprocessing

myModel = ANN_Model('ionosphere_data.csv')
myModel.cleanData()

In [97]:
myModel._RawCleanData

[[1.0,
  1.0,
  -0.18829,
  0.93035,
  -0.36156,
  -0.10868,
  -0.93597,
  1.0,
  -0.04549,
  0.50874,
  -0.67743,
  0.34432,
  -0.69707,
  -0.51685,
  -0.97515,
  0.05499,
  -0.62237,
  0.33109,
  -1.0,
  -0.13151,
  -0.453,
  -0.18056,
  -0.35734,
  -0.20332,
  -0.26569,
  -0.20468,
  -0.18401,
  -0.1904,
  -0.11593,
  -0.16626,
  -0.06288,
  -0.13738,
  -0.02447,
  'b'],
 [1.0,
  1.0,
  -0.45161,
  1.0,
  1.0,
  0.71216,
  -1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -1.0,
  0.14516,
  0.54094,
  -0.3933,
  -1.0,
  -0.54467,
  -0.69975,
  1.0,
  0.0,
  0.0,
  1.0,
  0.90695,
  0.51613,
  1.0,
  1.0,
  -0.20099,
  0.25682,
  1.0,
  -0.32382,
  1.0,
  'b'],
 [1.0,
  0.02337,
  -0.00592,
  -0.09924,
  -0.11949,
  -0.00763,
  -0.11824,
  0.14706,
  0.06637,
  0.03786,
  -0.06302,
  0.0,
  0.0,
  -0.04572,
  -0.1554,
  -0.00343,
  -0.10196,
  -0.11575,
  -0.05414,
  0.01838,
  0.03669,
  0.01519,
  0.00888,
  0.03513,
  -0.01535,
  -0.0324,
  0.09223,
  -0.07859,
  0.00732,
  0.0,
 

In [98]:
#myModel.MinMax_Normalization()

In [99]:
myModel._ScaledTrainData

[[1.0,
  1.0,
  -0.18829,
  0.93035,
  -0.36156,
  -0.10868,
  -0.93597,
  1.0,
  -0.04549,
  0.50874,
  -0.67743,
  0.34432,
  -0.69707,
  -0.51685,
  -0.97515,
  0.05499,
  -0.62237,
  0.33109,
  -1.0,
  -0.13151,
  -0.453,
  -0.18056,
  -0.35734,
  -0.20332,
  -0.26569,
  -0.20468,
  -0.18401,
  -0.1904,
  -0.11593,
  -0.16626,
  -0.06288,
  -0.13738,
  -0.02447],
 [1.0,
  1.0,
  -0.45161,
  1.0,
  1.0,
  0.71216,
  -1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -1.0,
  0.14516,
  0.54094,
  -0.3933,
  -1.0,
  -0.54467,
  -0.69975,
  1.0,
  0.0,
  0.0,
  1.0,
  0.90695,
  0.51613,
  1.0,
  1.0,
  -0.20099,
  0.25682,
  1.0,
  -0.32382,
  1.0],
 [1.0,
  0.02337,
  -0.00592,
  -0.09924,
  -0.11949,
  -0.00763,
  -0.11824,
  0.14706,
  0.06637,
  0.03786,
  -0.06302,
  0.0,
  0.0,
  -0.04572,
  -0.1554,
  -0.00343,
  -0.10196,
  -0.11575,
  -0.05414,
  0.01838,
  0.03669,
  0.01519,
  0.00888,
  0.03513,
  -0.01535,
  -0.0324,
  0.09223,
  -0.07859,
  0.00732,
  0.0,
  0.0,
  -0.000

In [100]:
len(myModel._ScaledTrainData)

351

In [101]:
# --- 3. Split Data --- 

myModel.splitData()

In [102]:
print("Input in matrix form :")
input_T = myModel._Input_Train.transpose()
print(input_T)

print("\nShape of Input Matrix:" , input_T._shape)

Input in matrix form :
[ 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 

In [103]:
myModel.hyperparameter()

No.Input Unit:  33
No.Hidden Layer:  1
No.Hidden Unit:  22
No.Output Unit:  1


### 5. เริ่มกระบวนการ Train
#### trainModel(seed, epoch , alpha , batch_size)

In [104]:
myModel.trainModel(42,200,0.001,32)

Random Seed : 42


Error at epoch 0 is 0.30462


KeyboardInterrupt: 

In [ ]:
myModel._weights_input_hidden

[ 0.08776457153715272, -0.14861465919872696, 0.32810103036307164, -0.03403203277137144, -0.25430281057408805, 0.20384536863793484, -0.2847583057078777, 0.4898643559374563, -0.0794626145696227, 0.27943249615223986, 0.8207291965639059, 0.5416973226685375, 0.602064756124105, -0.10734514830602174, 0.40311611591066215, 0.37725595575403614, 0.09091290168649374, -0.21489139520825215, -0.09652977430064322, -0.25454281067146345, 0.08467594327852206, -0.05808188764953246 ]
[ 0.1855638447180016, -0.5498849971701545, -0.2121545675242716, 0.07114426969128727, -0.14690462497781215, 0.2981863556696538, 0.11549132647476257, 0.48266831067414256, -0.13366368947556145, -0.0035434927344868443, 0.7373532565967448, -0.09532026247046288, -0.4018505905835014, 0.20401453513475115, -0.29141201505926867, 0.17293285733988617, -0.273838017483676, -0.25761409327935153, 0.0733626151381977, 0.18810746078876706, -0.005918763842351798, -0.238406160330665 ]
[ 0.03806797486499044, 0.23570942632145583, 0.2584634931122282,

In [ ]:
myModel._weights_hidden_output

[ 0.013368407051416568 ]
[ -0.47838323843029107 ]
[ -0.1404678622239234 ]
[ -0.23119037170180434 ]
[ -0.6712025727035515 ]
[ 0.6186796894833876 ]
[ -0.058131283907742505 ]
[ 0.2763453903794275 ]
[ -0.15584124581344383 ]
[ -0.9302327978107394 ]
[ 0.00918225311649514 ]
[ -0.045412442015630106 ]
[ -0.006906733343619477 ]
[ -0.15087607466227448 ]
[ 0.4056685970460221 ]
[ 0.07829185812072453 ]
[ -0.13844883413432169 ]
[ -0.3930307471306082 ]
[ -0.0084514860529415 ]
[ -0.2253671182307992 ]
[ -0.03433493050651412 ]
[ -0.3750122341456631 ]

In [ ]:
myModel.testModel()

Best Threshold is : 0.5959324183662162
Confusion Matrix (row=Actual, col=Predicted)
[[30, 12], [3, 72]]
Accuracy: 87.18%
Precision: 0.86
Recall: 0.96
F1 Score: 0.91
